# <a id='toc1_'></a>[JO 2024 project](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [JO 2024 project](#toc1_)    
  - [Prelude](#toc1_1_)    
  - [Imports](#toc1_2_)    
  - [Fonctions](#toc1_3_)    
  - [Data collect](#toc1_4_)    
    - [Extract one country data](#toc1_4_1_)    
    - [Extract All countries data](#toc1_4_2_)    
    - [Extract all data](#toc1_4_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Prelude](#toc0_)

Work in progress . . .  

Summarize about the project :
- Data source : <a href="https://www.olympic.org/news">International Olympic Committee</a>
- Data extract from : <a href="http://olympanalyt.com/OlympAnalytics.php">olympanalyt.com - mail : sportsencyclo@gmail.com</a>

## <a id='toc1_2_'></a>[Imports](#toc0_)

In [122]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## <a id='toc1_3_'></a>[Fonctions](#toc0_)

In [123]:
def GetHTML(url) :
    """
    Needs : url website
    Return : HTML text
    """
    response = requests.get(url)

    if response.status_code == 200 :
        print("Response OK, continue.")
    else :
        print("Access impossible.")
    
    return response.text

In [124]:
def Getdataframe(html) :

    """
    Needs : HTML text 
            with response = requests.get(url).text
    Return : Dataframe with athlete AND sport description : medal, country, athlete_name, games, sport, event
    Requirement : Must to apply on website like : adress[COUNTRY]adress
                i.e : http://olympanalyt.com/OlympAnalytics.php?param_pagetype=Medals&param_dbversion=&param_country=[COUNTRY]&param_games=ALL&param_sport=ALL
    """

    # Converting text (html) in soup (text parsed)
    soup = BeautifulSoup(html, "html5lib") #parse

    # Finding the correct table according to description
    table = soup.find("table", class_="simpletable", style="text-align:left;")

    # Retrieving the list (find_all) of all rows
    rows = table.find_all("tr")

    # Creating the table (future dataframe)
    athletes = []

    # For each row (= athlete)
    for row in rows:
        #describe_athlete = row.text.split() # Get the text
        cells = row.find_all("td") # Retrieving the list (find_all) of all cells in the row

        if len(cells) == 7: # If there are 7 cells then it's an athlete
            medal = cells[0].find('img')['title'][0]  # Medal title
            country = cells[1].find('img')['title']  # Athlete's country
            athlete_name = str(cells[2]).replace("<br/>","/").replace("</td>","").replace("<td>","") # Athlete's name, splitted by "/" if it's a team
            games = cells[3].find('img')['title']  # Olympic Games
            sport = cells[4].find('img')['title']  # Sport
            sex = cells[5].text.strip()  # Sex
            event = cells[6].text.strip()  # Event

            # Add the information to the list of athletes
            athletes.append({
                'medal': medal,
                'country': country,
                'athlete_name': athlete_name,
                'games': games,
                'sport': sport,
                'event': event,
                'sex' : sex
                })

        else: # Otherwise it's not an athlete, move to the next row
            next

    return pd.DataFrame(athletes)

url = "http://olympanalyt.com/OlympAnalytics.php?param_pagetype=Medals&param_dbversion=&param_country=FRA&param_games=ALL&param_sport=ALL"
# Try with one country - FRA (France)
html = GetHTML(url)

Getdataframe(html)

Response OK, continue.


,medal,country,athlete_name,games,sport,event,sex
0,1,France,Paul MASSON,Greece,Cycling Track,sprint individual,Men's
1,1,France,Paul MASSON,Greece,Cycling Track,1/3km time trial,Men's
2,1,France,Paul MASSON,Greece,Cycling Track,10km,Men's
3,1,France,Léon FLAMENG,Greece,Cycling Track,100km,Men's
4,1,France,Eugène-Henri GRAVELOTTE,Greece,Fencing,foil individual,Men's
...,...,...,...,...,...,...,...
884,2,France,SIMON Julia/FILLON MAILLET Quentin/CHEVALIER-B...,China,Biathlon,Relay mix,Mixed
885,2,France,LEDEUX Tess,China,Freestyle Skiing,Big air,Women's
886,2,France,TRESPEUCH Chloe,China,Snowboard,Snowboard Cross,Women's
887,3,France,FAIVRE Mathieu,China,Alpine Skiing,giant slalom,Men's


## <a id='toc1_4_'></a>[Data collect](#toc0_)

### <a id='toc1_4_1_'></a>[Extract one country data](#toc0_)

In [125]:
url = "http://olympanalyt.com/OlympAnalytics.php?param_pagetype=Medals&param_dbversion=&param_country=FRA&param_games=ALL&param_sport=ALL"

In [126]:
# Try with one country - FRA (France)
html = GetHTML(url)

Response OK, continue.


In [127]:
# Try to get the DF on one country - FRA (France)
df_medalists = Getdataframe(html)

df_medalists.sample(5)

,medal,country,athlete_name,games,sport,event,sex
458,2,France,Joseph MAHMOUD,United States,Athletics,3000m steeplechase,Men's
100,3,France,Auguste Albert/Charles Hugo/François Vilamitja...,France,Sailing,1-2t,Mixed
799,2,France,OUMIHA Sofiane,Brazil,Boxing,56 - 60kg (lightweight),Men's
544,3,France,Edgar GROSPIRON,Norway,Freestyle Skiing,moguls,Men's
494,3,France,Bruno MARIE-ROSE/Gilles QUENEHERVE/Daniel SANG...,Korea Republic,Athletics,4x100m relay,Men's


### <a id='toc1_4_2_'></a>[Extract All countries data](#toc0_)

Scrape countries codes to loop on each of them (website url modification)  
Link of countries codes : <a href="http://olympanalyt.com/OlympAnalytics.php?param_pagetype=RefCountries&param_dbversion=&param_country=CIV&param_games=ALL&param_sport=ALL"> Countries list</a>

In [128]:
# Countries list
url = "http://olympanalyt.com/OlympAnalytics.php?param_pagetype=RefCountries&param_dbversion=&param_country=CIV&param_games=ALL&param_sport=ALL"

html = GetHTML(url)

Response OK, continue.


In [129]:
# Parsing text in Beautifulsoup object
soup = BeautifulSoup(html, "html5lib")

In [130]:
# Select the right table
table = soup.find_all("table", class_="simpletable", style="text-align:center;")[1] # 2nd table
rows = table.find_all("tr")
countries = []

for row in rows[2:]: # ignore the column names
    data = row.text.split()

    if len(data) == 5 :
        countries.append(
            {
                'Code':data[0], 
                'Country':data[1], 
                'Continent':data[2], 
                'Firstparticipation':data[3], 
                'Lastparticipation':data[4]
            }
        )
    else :
        next

df_countries = pd.DataFrame(countries)

In [131]:
df_countries.sample(5)

,Code,Country,Continent,Firstparticipation,Lastparticipation
112,MGL,Mongolia,Asia,1964,2022
180,ZZX,Mixed,Team,1896,1904
174,as,"Rhodesia,",British,1928,1964
158,TOG,Togo,Africa,1972,2021
92,LAT,Latvia,Europe,1924,2022


### <a id='toc1_4_3_'></a>[Extract all data](#toc0_)

In [132]:
# Run scraping on all countries 
code_countries = df_countries["Code"]

i = 0
for country, country_name in zip(code_countries, df_countries["Country"]) :
    url = "http://olympanalyt.com/OlympAnalytics.php?param_pagetype=Medals&param_dbversion=&param_country=" + country + "&param_games=ALL&param_sport=ALL"
    html = GetHTML(url)
    df_temp = Getdataframe(html)
    if i == 0 :
        df_olympic = df_temp
        i += 1
    else : 
        df_olympic = pd.concat([df_olympic, df_temp], ignore_index=True)

    #Delete these codes after tries
    i += 1
    if i > 20 :
        break

    print("for : {}".format(country_name))

Response OK, continue.
for : Afghanistan
Response OK, continue.
for : Albania
Response OK, continue.
for : Algeria
Response OK, continue.
for : Andorra
Response OK, continue.
for : Angola
Response OK, continue.
for : Antigua,
Response OK, continue.
for : Argentina
Response OK, continue.
for : Armenia
Response OK, continue.
for : Aruba
Response OK, continue.
for : Australia
Response OK, continue.
for : Austria
Response OK, continue.
for : Azerbaijan
Response OK, continue.
for : Bahamas
Response OK, continue.
for : Bahrain
Response OK, continue.
for : Bangladesh
Response OK, continue.
for : Barbados
Response OK, continue.
for : Belarus
Response OK, continue.
for : Belgium
Response OK, continue.
for : Belize
Response OK, continue.


In [134]:
df_olympic.sample(15)

,medal,country,athlete_name,games,sport,event,sex
258,3,Australia,Mike WENDEN/Robert CUSACK/Gregory ROGERS/Rober...,Mexico,Swimming,4x100m freestyle relay,Men's
737,1,Austria,Herma BAUMA,Great Britain,Athletics,javelin throw,Women's
728,2,Austria,Fritz WURMBÖCK/Ferdinand KIEFLER/Franz BRUNNER...,Germany,Handball,handball,Men's
1082,1,Bahamas,Тоник Уильямс-Дарлинг,Greece,Athletics,400m,Women's
1028,1,Azerbaijan,Намиг Абдуллаев,Australia,Wrestling Freestyle,- 54kg,Men's
736,3,Austria,Max STIEPL,Germany,Speed skating,10000m,Men's
190,3,Australia,John LANDY,Australia / Sweden,Athletics,1500m,Men's
722,1,Austria,Gregor HRADETZKY,Germany,Canoe Sprint,folding K-1 10000m,Men's
913,3,Austria,Эберхартер,United States,Alpine Skiing,downhill,Men's
1067,3,Azerbaijan,SHARIATI Sabah,Brazil,Wrestling Greco-Roman,130kg,Men's
